In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [29]:
text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
beaut = BeautifulSoup(text, 'lxml')
wiki = beaut.find('table', class_='wikitable sortable')

table_list = []
for rows in wiki.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    table_list.append(row)

In [30]:
col_names = ['PostalCode', 'Borough', 'Neighbourhood'] 
dfcan=[]
dfcan=pd.DataFrame(columns=col_names)

In [31]:
dfcan.iloc[:,0]=table_list[::3]
dfcan.iloc[:,1]=table_list[1::3]
dfcan.iloc[:,2]=table_list[2::3]
dfcan.replace("Not assigned", np.nan, inplace = True)
dfcan.dropna(subset=["Borough"], axis=0, inplace = True)
dfcan.reset_index(drop=True, inplace=True)

In [32]:
for i in range(0, dfcan.shape[0]):
    if pd.isnull(dfcan.loc[i,'Neighbourhood']):
        dfcan.replace(dfcan.loc[i,'Neighbourhood'], dfcan.loc[i,'Borough'],inplace=True)
dfcan.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [33]:
dfcan['Neighbourhood'] = dfcan[['PostalCode','Borough','Neighbourhood']].groupby(['PostalCode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x)) 
dfcan.drop_duplicates(inplace=True)
dfcan.reset_index(drop=True, inplace=True)

dfcan.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [34]:
dfcan.shape

(103, 3)

In [35]:
df_longlat = pd.read_csv ("http://cocl.us/Geospatial_data")

In [36]:
df_longlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
